In [ ]:
import pandas as pd

class LL1:
    def __init__(self, grammar):
        self.grammar = grammar
        self.non_terminals = list(grammar.keys())
        self.terminals = self.get_terminals()
        self.first = self.compute_first()
        self.follow = self.compute_follow()
        self.table = self.construct_table()

    def get_terminals(self):
        terminals = set()
        for productions in self.grammar.values():
            for prod in productions:
                for char in prod:
                    if not char.isupper():
                        terminals.add(char)
        return list(terminals)

    def compute_first(self):
        first = {}
        for non_terminal in self.non_terminals:
            first[non_terminal] = list()

        for non_terminal in self.non_terminals:
            first[non_terminal].append(self.find_first(non_terminal))
        return first

    def find_first(self, non_terminal):
        productions = self.grammar[non_terminal]
        
        for production in productions:
            first_element = production[0]

            if first_element.isupper():
                return self.find_first(first_element)
            else:
                return first_element
    # Follow()
    def compute_follow(self):
        follow = {i: set() for i in self.non_terminals}
        follow[self.non_terminals[0]] = {'$'}
        
        for non_terminal in self.non_terminals:
            self.find_follow(non_terminal, follow)
        
        return follow


    def find_follow(self, non_terminal, follow):
        for key, productions in self.grammar.items():
            for prod in productions:
                if non_terminal in prod:
                    next_symbols = prod[prod.index(non_terminal) + 1:]
                    if key == non_terminal:
                        continue
                    else:
                        if next_symbols == '':
                            follow[non_terminal] = follow[key]
                            break
                        else:
                            if not next_symbols.isupper():
                                follow[non_terminal].add(next_symbols)
                                break
                            else:
                                follow[non_terminal] = follow[non_terminal].union(self.first[next_symbols[0]])
                                break
        return follow[non_terminal]

    def construct_table(self):
        firsts = list(self.first.values())

        tempFirsts = list()
        for value in firsts:
            tempFirsts.append(value[0])
        
        table = pd.DataFrame(
                {
                    'Non Terminal': self.non_terminals,
                    'First()': tempFirsts,
                    'Follow()': list(self.follow.values())
                }
            )
        return table
    
    def display_table(self):
        print(self.table)    

grammar = {
    'E': ['TX'],
    'X': ['+TX'],
    'T': ['-FY='],
    'Y': ['*FY'],
    'F': ['(E)']
}

ll1_parser = LL1(grammar)
ll1_parser.construct_table()


,Non Terminal,First(),Follow()
0,E,-,"{), $}"
1,X,+,"{), $}"
2,T,-,{+}
3,Y,*,{=}
4,F,(,{*}
